This notebook reformats the energy consumption, generation & pricing data sourced from the EPİAŞ transparency platform ([link](https://seffaflik.epias.com.tr/home)).

## Setup

In [6]:
import pandas as pd
import numpy as np

In [7]:
input_dir = "./InputData/"
output_dir = "./OutputData/"

## Data prep, consumption data

In [12]:
# Load & concatenate data
df_consumption = pd.concat([
        pd.read_csv(input_dir + "Real_Time_Consumption20.csv", sep = ";"),
        pd.read_csv(input_dir + "Real_Time_Consumption23.csv", sep = ";")
    ])

# Reindex rows, rename columns
df_consumption = df_consumption.reset_index(drop = True)
df_consumption = df_consumption.rename({
    "Tarih": "date",
    "Saat": "hour",
    "Tüketim Miktarı(MWh)": "consumption_MWh"
}, axis = 1)

In [13]:
# Combine date & hour
df_consumption["time"] = df_consumption["date"].str.replace(".", ":") + ":" + df_consumption["hour"]
df_consumption = df_consumption.drop(["date", "hour"], axis = 1)

In [14]:
# Remove dots indicating thousands
df_consumption["consumption_MWh"] = df_consumption["consumption_MWh"].str.replace(".", "")

# Change fractional commas to dots, change consumption to float
df_consumption["consumption_MWh"] = df_consumption["consumption_MWh"].str.replace(",", ".").astype(float)

In [15]:
df_consumption

,consumption_MWh,time
0,27412.81,01:01:2018:00:00
1,26324.39,01:01:2018:01:00
2,24635.32,01:01:2018:02:00
3,23872.12,01:01:2018:03:00
4,23194.89,01:01:2018:04:00
...,...,...
52579,35090.93,31:12:2023:19:00
52580,33310.94,31:12:2023:20:00
52581,32083.96,31:12:2023:21:00
52582,30469.49,31:12:2023:22:00


In [16]:
# Check missing values
pd.isna(df_consumption).sum()

consumption_MWh    0
time               0
dtype: int64

In [17]:
# Check unusual values
df_consumption.describe()

,consumption_MWh
count,52584.000000
mean,35166.891649
std,5743.832244
min,15333.300000
25%,30814.882500
50%,35063.730000
75%,39281.595000
max,55575.020000


## Data prep, generation data

In [18]:
# Load 2018 data
df_generation = pd.read_csv(input_dir + "Real_Time_Generation18.csv", sep = ";")

# Load & concatenate rest of data
for year in range(19, 24, 1):
    df = pd.read_csv(input_dir + f"Real_Time_Generation{year}.csv", sep = ";")
    df_generation = pd.concat([df_generation, df])

In [19]:
newnames = [
    "date", "hour", "total_generation_MWh", "natural_gas", "hydro_dam", "lignite", 
    "hydro_river", "coal_imported", "wind", "solar", "fuel_oil", "geothermal", 
    "asphaltite_coal", "hard_coal", "biomass", "naphtha", "LNG", "international", 
    "waste_heat"]

In [20]:
# Reindex, rename columns
df_generation = df_generation.reset_index(drop = True)
df_generation = df_generation.rename(
    dict(zip(df_generation.columns, newnames)), axis = 1)

In [21]:
# Combine date & hour
df_generation["time"] = df_generation["date"].str.replace(".", ":") + ":" + df_generation["hour"]
df_generation = df_generation.drop(["date", "hour"], axis = 1)

In [22]:
# Change fractional commas to dots
df_generation = df_generation.apply(
    lambda col: col.astype(str), axis = 1)

df_generation.iloc[:, :-1] = df_generation.iloc[:, :-1].apply(
    lambda col: col.str.replace(",", "."), 
    axis = 1)

# Change datatypes
newtypes = ("float " * 17).split(" ")[0:-1]
newtypes.append("object")
df_generation = df_generation.astype(
    dict(zip(df_generation.columns, newtypes)))

In [23]:
df_generation

,total_generation_MWh,natural_gas,hydro_dam,lignite,hydro_river,coal_imported,wind,solar,fuel_oil,geothermal,asphaltite_coal,hard_coal,biomass,naphtha,LNG,international,waste_heat,time
0,27412.81,6926.78,4820.40,5226.32,1900.32,5314.05,1179.51,0.0,176.60,801.09,278.20,189.50,210.96,0.0,0.0,297.00,92.08,01:01:2018:00:00
1,26324.39,6603.44,3644.32,5277.19,1877.83,5715.61,1148.46,0.0,174.20,803.21,284.82,205.50,211.67,0.0,0.0,287.00,91.14,01:01:2018:01:00
2,24635.32,6040.77,2711.00,5325.47,1708.49,5701.37,1216.38,0.0,177.91,804.28,278.20,201.50,209.89,0.0,0.0,169.00,91.06,01:01:2018:02:00
3,23872.12,5938.46,2524.37,5265.91,1662.59,5205.43,1298.11,0.0,171.50,805.61,271.57,211.50,211.24,0.0,0.0,214.00,91.83,01:01:2018:03:00
4,23194.89,5665.41,2462.71,5282.88,1697.96,4804.19,1220.91,0.0,165.00,806.01,273.78,307.50,214.31,0.0,0.0,203.00,91.23,01:01:2018:04:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52579,34753.40,5567.82,8719.86,5311.18,2287.36,9100.92,359.70,0.0,93.36,1219.49,130.27,613.05,997.70,0.0,0.0,270.20,82.49,31:12:2023:19:00
52580,31441.31,4792.96,7881.64,4908.48,2083.33,7613.93,459.87,0.0,93.46,1254.10,132.48,612.95,983.26,0.0,0.0,539.84,85.01,31:12:2023:20:00
52581,30368.98,4000.94,7513.96,4911.94,1966.32,7656.56,552.62,0.0,88.76,1246.25,132.48,607.59,977.02,0.0,0.0,629.91,84.63,31:12:2023:21:00
52582,30386.41,2903.52,8119.63,4615.93,1837.75,8390.47,720.11,0.0,83.06,1278.77,130.27,608.31,987.47,0.0,0.0,627.77,83.35,31:12:2023:22:00


In [24]:
# Check missing values
pd.isna(df_generation).sum()

total_generation_MWh    0
natural_gas             0
hydro_dam               0
lignite                 0
hydro_river             0
coal_imported           0
wind                    0
solar                   0
fuel_oil                0
geothermal              0
asphaltite_coal         0
hard_coal               0
biomass                 0
naphtha                 0
LNG                     0
international           0
waste_heat              0
time                    0
dtype: int64

In [25]:
# Check unusual values
df_generation.describe()

,total_generation_MWh,natural_gas,hydro_dam,lignite,hydro_river,coal_imported,wind,solar,fuel_oil,geothermal,asphaltite_coal,hard_coal,biomass,naphtha,LNG,international,waste_heat
count,52584.000000,52584.000000,52584.000000,52584.000000,52584.000000,52584.000000,52584.000000,52584.000000,52584.000000,52584.000000,52584.000000,52584.000000,52584.000000,52584.0,52584.000000,52584.000000,52584.000000
mean,34399.243138,8725.505190,5614.316871,4901.607249,2219.311611,7128.778313,3137.646356,167.942769,83.699436,1044.447744,235.883465,415.177407,590.215587,0.0,0.018057,52.954873,81.734692
std,5272.179656,4027.024572,2846.858883,616.084315,1333.785556,1823.970761,1910.953417,340.820824,54.734514,177.130975,82.835939,115.472528,251.158824,0.0,0.167291,360.610536,13.671189
min,15333.300000,496.660000,296.690000,2189.100000,379.960000,956.350000,42.530000,-1.700000,0.000000,525.110000,0.000000,77.500000,202.580000,0.0,0.000000,-3130.500000,30.690000
25%,30485.697500,5648.690000,3333.605000,4555.622500,1140.782500,5931.242500,1577.745000,0.000000,37.000000,916.442500,196.500000,331.500000,359.625000,0.0,0.000000,-216.617500,73.120000
50%,34438.515000,8863.035000,5362.740000,4946.670000,1725.255000,7604.270000,2817.655000,3.510000,71.900000,1068.180000,231.830000,415.015000,543.140000,0.0,0.000000,0.000000,82.140000
75%,38188.935000,11551.372500,7667.975000,5314.530000,3167.205000,8456.725000,4403.705000,115.470000,98.162500,1191.620000,320.150000,510.540000,852.662500,0.0,0.000000,333.000000,91.220000
max,51285.690000,19453.650000,15891.780000,6984.150000,6103.590000,11352.140000,9737.590000,1547.410000,237.600000,1458.280000,368.720000,692.620000,1039.900000,0.0,1.880000,1648.540000,127.760000


In [26]:
# Check if the total adds up
(df_generation["total_generation_MWh"] - df_generation.iloc[:, 1:-1].sum(axis = 1)).describe()

count    5.258400e+04
mean     3.518941e-03
std      2.182045e-01
min     -2.182787e-11
25%     -3.637979e-12
50%      0.000000e+00
75%      3.637979e-12
max      2.317000e+01
dtype: float64

## Data prep, market clearing price data

In [27]:
# Load & concatenate data
df_price = pd.concat([
    pd.read_csv(input_dir + "Market_Clearing_Price20.csv", sep = ";"),
    pd.read_csv(input_dir + "Market_Clearing_Price23.csv", sep = ";")
])

# Reindex rows & rename columns
df_price = df_price.reset_index(drop = True)
df_price = df_price.rename({
    "Tarih": "date",
    "Saat": "hour",
    "PTF (TL/MWh)": "TRY/MWh",
    "PTF (USD/MWh)": "USD/MWh",
    "PTF (EUR/MWh)": "EUR/MWh"
}, axis = 1)

In [28]:
# Combine date & hour
df_price["time"] = df_price["date"].str.replace(".", ":") + ":" + df_price["hour"]
df_price = df_price.drop(["date", "hour"], axis = 1)

In [29]:
# Remove dots indicating thousands
df_price.iloc[:, :-1] = df_price.iloc[:, :-1].apply(
    lambda col: col.str.replace(".", ""), 
    axis = 1)

# Change fractional commas to dots, change price to float
df_price.iloc[:, :-1] = df_price.iloc[:, :-1].apply(
    lambda col: col.str.replace(",", "."), 
    axis = 1)

# Change datatypes
newtypes = ("float " * 3).split(" ")[0:-1]
newtypes.append("object")
df_price = df_price.astype(
    dict(zip(df_price.columns, newtypes)))

In [30]:
df_price

,TRY/MWh,USD/MWh,EUR/MWh,time
0,207.60,55.04,45.97,01:01:2018:00:00
1,205.34,54.44,45.47,01:01:2018:01:00
2,164.94,43.73,36.53,01:01:2018:02:00
3,154.52,40.97,34.22,01:01:2018:03:00
4,112.64,29.86,24.95,01:01:2018:04:00
...,...,...,...,...
52579,2499.67,84.91,76.74,31:12:2023:19:00
52580,2472.34,83.98,75.90,31:12:2023:20:00
52581,2472.33,83.98,75.90,31:12:2023:21:00
52582,1800.00,61.15,55.26,31:12:2023:22:00


In [31]:
# Check missing values
pd.isna(df_price).sum()

TRY/MWh    0
USD/MWh    0
EUR/MWh    0
time       0
dtype: int64

In [32]:
# Check unusual values
df_price.describe()

,TRY/MWh,USD/MWh,EUR/MWh
count,52584.000000,52584.000000,52584.000000
mean,996.139021,72.190211,65.793224
std,1126.233416,49.815156,49.431674
min,0.000000,0.000000,0.000000
25%,280.000000,42.850000,36.090000
50%,326.815000,54.350000,48.210000
75%,1699.000000,84.920000,78.012500
max,4800.000000,264.170000,271.630000


## Merge & export

In [33]:
# Merge dataframes, change column order
df = df_consumption.merge(df_generation, on = "time", how = "inner").merge(df_price, on = "time", how = "inner")
cols = np.delete(df.columns.values, 1).tolist()
cols.insert(0, "time")
df = df[cols]

In [34]:
df

,time,consumption_MWh,total_generation_MWh,natural_gas,hydro_dam,lignite,hydro_river,coal_imported,wind,solar,...,asphaltite_coal,hard_coal,biomass,naphtha,LNG,international,waste_heat,TRY/MWh,USD/MWh,EUR/MWh
0,01:01:2018:00:00,27412.81,27412.81,6926.78,4820.40,5226.32,1900.32,5314.05,1179.51,0.0,...,278.20,189.50,210.96,0.0,0.0,297.00,92.08,207.60,55.04,45.97
1,01:01:2018:01:00,26324.39,26324.39,6603.44,3644.32,5277.19,1877.83,5715.61,1148.46,0.0,...,284.82,205.50,211.67,0.0,0.0,287.00,91.14,205.34,54.44,45.47
2,01:01:2018:02:00,24635.32,24635.32,6040.77,2711.00,5325.47,1708.49,5701.37,1216.38,0.0,...,278.20,201.50,209.89,0.0,0.0,169.00,91.06,164.94,43.73,36.53
3,01:01:2018:03:00,23872.12,23872.12,5938.46,2524.37,5265.91,1662.59,5205.43,1298.11,0.0,...,271.57,211.50,211.24,0.0,0.0,214.00,91.83,154.52,40.97,34.22
4,01:01:2018:04:00,23194.89,23194.89,5665.41,2462.71,5282.88,1697.96,4804.19,1220.91,0.0,...,273.78,307.50,214.31,0.0,0.0,203.00,91.23,112.64,29.86,24.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52579,31:12:2023:19:00,35090.93,34753.40,5567.82,8719.86,5311.18,2287.36,9100.92,359.70,0.0,...,130.27,613.05,997.70,0.0,0.0,270.20,82.49,2499.67,84.91,76.74
52580,31:12:2023:20:00,33310.94,31441.31,4792.96,7881.64,4908.48,2083.33,7613.93,459.87,0.0,...,132.48,612.95,983.26,0.0,0.0,539.84,85.01,2472.34,83.98,75.90
52581,31:12:2023:21:00,32083.96,30368.98,4000.94,7513.96,4911.94,1966.32,7656.56,552.62,0.0,...,132.48,607.59,977.02,0.0,0.0,629.91,84.63,2472.33,83.98,75.90
52582,31:12:2023:22:00,30469.49,30386.41,2903.52,8119.63,4615.93,1837.75,8390.47,720.11,0.0,...,130.27,608.31,987.47,0.0,0.0,627.77,83.35,1800.00,61.15,55.26


In [35]:
# Export full data
df.to_csv(output_dir + "full_data.csv", index = False)